In [106]:
import os
import sqlalchemy
import pandas as pd
from dotenv import load_dotenv 
from scipy.sparse import coo_matrix
from sqlalchemy import create_engine, text
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from implicit.evaluation import train_test_split, precision_at_k, AUC_at_k

In [107]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

try:
    connection = engine.connect()
    print("Successfully connected to the database")
except Exception as e:
    print(f"Failed to connect to the database: {e}")

print(f"SQLAlchemy version: {sqlalchemy.__version__}")

Successfully connected to the database
SQLAlchemy version: 2.0.21


In [108]:
query = text('SELECT * FROM epic_5')
df = pd.read_sql_query(query, connection)
df.head()

,PersoonId,CampagneId,aantal_sessies,aantal_bezoeken,SessieThema,SoortCampagne,TypeCampagne,ThemaDuurzaamheid,ThemaFinancieelFiscaal,ThemaInnovatie,ThemaInternationaalOndernemen,ThemaMobiliteit,ThemaOmgeving,ThemaSalesMarketingCommunicatie,ThemaStrategieEnAlgemeenManagement,ThemaTalent,ThemaWelzijn
0,A03EC8AF-ED66-EB11-8119-001DD8B72B61,186D9250-435B-EB11-811A-001DD8B72B62,1,0,Algemeen Management,Online,Infosessie,0,0,0,0,0,0,0,0,0,0
1,78CCAFF0-617A-E611-80DE-001DD8B72B61,49E02605-AD7A-E811-80F3-001DD8B72B61,1,0,Marketing & Sales,Offline,Opleiding,0,0,0,0,0,0,0,0,0,0
2,1BD50DEE-F867-E111-A00F-00505680000A,4A05F086-EA3F-E911-80FC-001DD8B72B62,1,0,Internationaal Ondernemen,Offline,Opleiding,1,1,1,1,0,0,0,1,1,0
3,14D55A3B-7732-E311-8A47-005056B06EC4,6B584E91-903A-E911-80FC-001DD8B72B61,4,0,Communicatie,Offline,Opleiding,0,0,0,0,0,0,0,0,0,0
4,DBEA757F-0B68-E111-A00F-00505680000A,C6D72260-E451-EC11-8C62-000D3ABFCF4A,1,1,Groeien,Online,Netwerkevenement,0,0,0,0,0,0,0,0,0,0


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52567 entries, 0 to 52566
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   PersoonId                           52567 non-null  object
 1   CampagneId                          52567 non-null  object
 2   aantal_sessies                      52567 non-null  int64 
 3   aantal_bezoeken                     52567 non-null  int64 
 4   SessieThema                         52567 non-null  object
 5   SoortCampagne                       52567 non-null  object
 6   TypeCampagne                        52567 non-null  object
 7   ThemaDuurzaamheid                   52567 non-null  int64 
 8   ThemaFinancieelFiscaal              52567 non-null  int64 
 9   ThemaInnovatie                      52567 non-null  int64 
 10  ThemaInternationaalOndernemen       52567 non-null  int64 
 11  ThemaMobiliteit                     52567 non-null  in

SOURCE: https://benfred.github.io/implicit/index.html

In [110]:
# toch groeperen op aantal sessies per persoon per campagne, geen scheiding tussen sessiethema
df = df.groupby(['PersoonId', 'CampagneId'])['aantal_sessies'].sum().reset_index()

# persoonid en campagneid omzetten naar integers voor de sparse matrix invoer van implicit
persoon_mapping = {id: i for i, id in enumerate(df['PersoonId'].unique())}
campagne_mapping = {id: i for i, id in enumerate(df['CampagneId'].unique())}

# reverse mapping voor later
persoon_reverse_mapping = {i: id for id, i in persoon_mapping.items()}
campagne_reverse_mapping = {i: id for id, i in campagne_mapping.items()}

# mapping toepassen op df
df['PersoonId'] = df['PersoonId'].map(persoon_mapping)
df['CampagneId'] = df['CampagneId'].map(campagne_mapping)

In [111]:
df.head()

,PersoonId,CampagneId,aantal_sessies
0,0,0,1
1,0,1,1
2,1,2,1
3,2,3,1
4,3,4,1


In [112]:
# sparse matrix maken met aantal sessies als waarden
campagne_persoon_sessies = coo_matrix((df['aantal_sessies'], (df['CampagneId'], df['PersoonId'])))

In [113]:
# matrix wegen, om hoge aantal sessies minder zwaar te laten wegen
campagne_persoon_sessies = bm25_weight(campagne_persoon_sessies, K1=100, B=0.5)

# matrix transponeren omdat implicit werkt met (user, item) in plaats van (item, user)
persoon_sessies = campagne_persoon_sessies.T.tocsr()

In [114]:
trainset, testset = train_test_split(persoon_sessies, train_percentage=0.8, random_state=42)

In [115]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [116]:
model = AlternatingLeastSquares(factors=128, regularization=0.15, alpha=5.0, iterations=100, random_state=42)
model.fit(trainset)

  1%|          | 1/100 [00:00<01:29,  1.11it/s]

100%|██████████| 100/100 [01:23<00:00,  1.20it/s]


In [117]:
precision = precision_at_k(model, trainset, testset, K=10, num_threads=4)
auc = a = AUC_at_k(model, trainset, testset, K=10, num_threads=4)

print(f"Precision: {precision}", f"AUC: {auc}", sep="\n")

100%|██████████| 6023/6023 [00:00<00:00, 8690.88it/s]

Precision: 0.29319591965506364
AUC: 0.6329666986372277


In [118]:
def get_similar_items(item_id):
    similar = model.similar_items(item_id, 10+1) 
    return similar[0][1:]

# get_similar_items(5)

In [119]:
def get_similar_users(user_id):
    similar = model.similar_users(user_id, 10+1) 
    return similar[0][1:]
    
# get_similar_users(5)

In [120]:
# WORK IN PROGRESS UNDER THIS LINE

In [121]:
# model recommend
# https://benfred.github.io/implicit/api/models/cpu/als.html#implicit.cpu.als.AlternatingLeastSquares.user_factors

In [122]:
# combineren met recommend van het model
def get_top_users_for_item(string_item_id, N=20):

    item_id = campagne_mapping[string_item_id]

    similar_items = get_similar_items(item_id)

    user_ids = []
    for similar_item in similar_items:
        users_for_item = df[df['CampagneId'] == similar_item]['PersoonId'].unique()
        user_ids.extend(users_for_item)

    user_ids = list(set(user_ids))

    users_for_input_item = df[df['CampagneId'] == item_id]['PersoonId'].unique()
    user_ids = [user_id for user_id in user_ids if user_id not in users_for_input_item]

    string_user_ids = [persoon_reverse_mapping[user_id] for user_id in user_ids]

    return string_user_ids[:N]

In [123]:
get_top_users_for_item('A534E7FC-EEF3-E711-80EE-001DD8B72B62')

['1E7BDC7A-A018-E811-80EF-001DD8B72B61',
 '9E3D74E5-7257-E811-80F0-001DD8B72B62',
 '00CBD1B2-A70D-E811-80EE-001DD8B72B62',
 '1F3E2F81-BD01-E811-80EF-001DD8B72B61',
 '1F7C2C5D-B40D-E511-ABE8-005056B06EB4',
 'FCD7F0F5-21B7-E111-A45C-00505680000A',
 '9EE1126D-1368-E111-A00F-00505680000A',
 '0146B8DB-824A-E711-80E7-001DD8B72B61',
 '7F0A42E3-6F69-E111-B43A-00505680000A',
 '9EFFEA29-B1DF-E511-A303-005056B06EC4',
 '5DC3D3CA-5AB7-E411-9B05-005056B06EB4',
 '3F728C00-9BDA-E711-80EE-001DD8B72B61',
 '01A6A7EE-6469-E111-B43A-00505680000A',
 '01E08C1A-517E-E611-80DE-001DD8B72B61',
 '5E20E7B3-E767-E111-A00F-00505680000A',
 '3FF82ED5-801C-E811-80F0-001DD8B72B61',
 '9FE06B99-B83C-E411-9EE6-005056B06EC4',
 '5E8723B6-1C68-E111-A00F-00505680000A',
 '02821986-8F44-E411-8F25-005056B06EB4',
 '5F19A8DA-4969-E111-B43A-00505680000A']

In [124]:
# recommend (omvormen functie), model opslaan
# wat te doen met de campagnesessieinformatie? (populaire campagnes recommenden aan nieuwe gebruikers?)

Handling Cold Start Issues:

    Address the "cold start" problem for new users or items by incorporating demographic information, leveraging item features, or using popularity-based recommendations until sufficient user interactions are available.

Post-Processing Techniques:

    Apply post-processing techniques, such as re-ranking or filtering, to refine the recommendations based on additional criteria or business rules.

In [125]:
# WORK IN PROGRESS ABOVE THIS LINE

In [126]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

print ("Model exported successfully")

Model exported successfully
